### Import Packages

In [1]:
# work with geospatial data
import geopandas as gpd
import pandas as pd

# work with files
import yaml

# work with time
import pytz

# util functions
from geospatial_utils import *

### Essential Variables

In [4]:
# essential variables

# let's try to convert everything to EPSG:5070. For more accurate area measurements

# boundary for state of California
us_states = gpd.read_file("Data/Boundaries/cb_2018_us_state_500k/cb_2018_us_state_500k.shp")
us_states.to_crs("EPSG:5070", inplace=True)
ca_state = us_states[us_states["STUSPS"] == "CA"]

# HUC8 subbasins
huc8 = gpd.read_file("Data/Boundaries/HUC8_CONUS/HUC8_US.shp")
huc8.to_crs("EPSG:5070", inplace=True)
huc8['CA'] = huc8["STATES"].map(lambda x: "CA" in x)
huc8 = huc8[huc8["CA"]]

# intersect with California
huc8_ca = gpd.clip(huc8, ca_state)

Convert electricity prices to HUC8 subbasin level

In [7]:
huc8_ca

,TNMID,METASOURCE,SOURCEDATA,SOURCEORIG,SOURCEFEAT,LOADDATE,GNIS_ID,AREAACRES,AREASQKM,STATES,HUC8,NAME,Shape_Leng,Shape_Area,geometry,CA
147,{B76D3BD6-0284-4214-B645-E164287FF0D9},None,None,None,None,2012/06/11,0,986669.08,3992.91,"AZ,CA,MX",15030107,Lower Colorado,5.596144,0.382127,"POLYGON ((-1727436.233 1270389.252, -1727421.8...",True
371,{843EF822-4013-48BF-B25F-491A1CC8536A},None,None,None,None,2018/04/20,0,1100035.15,4451.69,"CA,MX",18070305,Cottonwood-Tijuana,4.460922,0.427042,"POLYGON ((-1889665.056 1299715.823, -1889658.2...",True
384,{68E4B12C-560E-48E0-8305-49C2EF033B13},None,None,None,None,2018/04/20,0,418280.85,1692.72,"CA,MX",18100202,Carrizo Creek,3.312948,0.163019,"POLYGON ((-1830181.014 1290146.484, -1830192.6...",True
386,{2983B57A-C017-4813-9063-51B4213FF9AB},None,None,None,None,2018/02/12,0,3205929.12,12973.95,"CA,MX",18100204,Salton Sea,8.257350,1.251881,"POLYGON ((-1771125.18 1342228.3, -1771129.428 ...",True
370,{8793F8E0-D35A-4E65-ABBB-FBFA84D43675},None,None,None,None,2012/06/11,0,993894.72,4022.15,"CA,MX",18070304,San Diego,3.496308,0.387564,"MULTIPOLYGON (((-1895876.115 1328068.767, -189...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,{0AD32C24-FFE3-494A-9709-35BDFD9415B8},None,None,None,None,2012/06/11,0,980127.18,3966.44,"CA,OR",18010209,Lower Klamath,4.895733,0.427944,"POLYGON ((-2219156.326 2432389.358, -2219153.7...",True
1039,{E8EB0B22-996C-4AB6-8B2A-B7072705DE1E},None,None,None,None,2012/06/11,0,627600.85,2539.81,"CA,OR",18010101,Smith,3.260496,0.275191,"POLYGON ((-2253750.512 2442308.485, -2253764.1...",True
1208,{14F46837-E6A9-40D4-9C77-3DF243AFF458},None,None,None,None,2013/01/18,0,493047.70,1995.30,"CA,OR",17100309,Applegate,2.996181,0.217376,"POLYGON ((-2183171.269 2422349.128, -2183184.4...",True
1209,{A3897E42-3682-4035-B7AA-BD349A3ED74C},None,None,None,None,2013/01/18,0,633550.82,2563.89,"CA,OR",17100311,Illinois,3.479440,0.279768,"POLYGON ((-2231170.966 2436127.81, -2231184.12...",True


In [15]:
ca_county_prices = gpd.read_file("Data/Boundaries/ca_counties.geojson")

ca_county_prices = ca_county_prices[~ca_county_prices["Electricity Price ($/MWh)"].isna()]

# convert electricity prices from county level to HUC8 level data
prices_county_to_huc8 = resolve_regions(ca_county_prices, huc8_ca, "Electricity Price ($/MWh)", "HUC8")

prices_county_to_huc8.convert_regions()

Compute solar capacity factor from DNI, DHI, GHI

In [ ]:
ca_solar = np.load("Data/Solar/ca_irradiation.npz")

time_index = ca_solar['time_index']
time_index.tz_localize(pytz.utc)

delta_time = (time_index[1] - time_index[0]).total_seconds()/3600 # time step, in hours
agg_number = 24/delta_time # number of time steps to aggregate to get a full day
dni_subbasins = ca_solar['dni']
dhi_subbasins = ca_solar['dhi']
ghi_subbasins = ca_solar['ghi']

In [ ]:
# calculate the beta angle at each point.

CA_time = pytz.timezone('America/Los_Angeles')

beta_angles = np.zeros((time_index.shape[0], huc8_ca.shape[0]))

azimuth_angles = np.zeros((time_index.shape[0], huc8_ca.shape[0]))

# get local clock time
curr_times = time_index.map(lambda x: x.astimezone(CA_time))

for idx, (_, row) in enumerate(huc8_ca.iterrows()):
    curr_betas = []
    curr_azimuths = []
    
    centroid_lat, centroid_lon = row['centroid_lat_lon']

    # longitude correction. local meridian is 120 degrees for California
    longitude_add = 4*(120 - centroid_lon)

    for stamp in curr_times:
        # get current hour and day
        hour = stamp.timetuple().tm_hour + (1/60)*stamp.timetuple().tm_minute
        day = stamp.timetuple().tm_yday
               
        # adding factor: equation of time
        time_input = (360/364)*(day-81)
        E = 9.87*np.sin(2*time_input*(2*np.pi/360)) - 7.53*np.cos(time_input*(2*np.pi/360)) - 1.5*np.sin(time_input*(2*np.pi/360))
        
        # get the hour angle
        H = 15*(12 - (hour + longitude_add/60 + E/60))
        
        declination = 23.45*np.sin((360/365)*(day - 81)*(2*np.pi/360))

        sin_beta = np.cos(centroid_lat*(2*np.pi/360))*np.cos(declination*(2*np.pi/360))*np.cos(H*(2*np.pi/360)) + np.sin(centroid_lat*(2*np.pi/360))*np.sin(declination*(2*np.pi/360))

        beta = np.arcsin(sin_beta)*(360/(2*np.pi))

        curr_betas.append(beta)

        sin_azimuth = (np.cos(declination*(2*np.pi/360))*np.sin(H*(2*np.pi/360)))/np.cos(beta*(2*np.pi/360))

        azimuth = np.arcsin(sin_azimuth)*(360/(2*np.pi))

        curr_azimuths.append(azimuth)

    beta_angles[:, idx] = np.array(curr_betas) # in degrees

    azimuth_angles[:, idx] = np.array(curr_azimuths) # in degrees

In [ ]:
# putting it all together to calculate solar energy production

cos_theta = np.sqrt(1 - (np.cos(beta_angles*(2*np.pi/360)) * np.cos(azimuth_angles*(2*np.pi/360)))**2)

rho = 0.2 # a typical value for ground reflectivity

I_BC = dni_subbasins * cos_theta # beam insolation on collector

I_DC = dhi_subbasins * ((1 + (np.sin(beta_angles*(2*np.pi/360)) / cos_theta))/2) # diffuse insolation on collector

I_RC = ghi_subbasins * rho * ((1 - (np.sin(beta_angles*(2*np.pi/360)) / cos_theta))/2) # reflected insolation on collector

total_insolation = I_BC + I_DC + I_RC

In [ ]:
# aggregate the insolation values into days
total_insolation = total_insolation.reshape((ghi_subbasins.shape[0]//agg_number, agg_number, huc8_ca.shape[0])) # shape is (# days, # time steps in a day, # HUC8 subbasins)

# sum across axis 1 (the time steps in the day) to get daily insolation
total_insolation = total_insolation.sum(axis=1)*delta_time # W/(m^2 * day).

In [ ]:
# save solar data
insolation_df = pd.DataFrame(total_insolation)
insolation_df.to_csv("Data/Solar/insolation_CA.csv") # W/(m^2 * day)

In [ ]:
# now think about generation per watt of capacity, per day
insolation_df = pd.read_csv("Data/Solar/insolation_CA.csv", index_col=0)
insolation_df_per_watt = insolation_df/1000 # TODO verify this is correct. we divide by 1000 to get a per-watt measurement. https://www.greenlancer.com/post/solar-panel-wattage-output-explained#:~:text=A%20solar%20panel%20rating%20measures,sunlight%20at%201000W%2Fsquare%20meters.
insolation_df_per_watt.to_csv("Data/Solar/insolation_CA_per_watt.csv")

Convert Wind Speed to Wind Power

In [ ]:
ca_wind = np.load("Data/Wind/wind_speed_subbasin_CA.csv")

In [ ]:
# read data for GE1.5-77 wind turbine model. Source: https://nrel.github.io/turbine-models/DOE_GE_1.5MW_77.html

# power rating curve
GE_power_curve = pd.read_csv('Data/Wind/DOE_GE_1.5MW_77.csv')

rated_wind_speeds = GE_power_curve['Wind Speed [m/s]'].values
rated_wind_power = GE_power_curve['Power [kW]'].values

# other specs
with open("Data/Wind/DOE_GE_1.5MW_77.yaml", 'r') as f:
    turbine_specs = yaml.safe_load(f)
    cut_in = turbine_specs['cut_in_wind_speed']
    cut_out = turbine_specs['cut_out_wind_speed']
    rated_wind = turbine_specs['rated_wind_speed']
    rated_power = turbine_specs['rated_power']

In [ ]:
def wind_speed_to_energy(wind_speeds):
    """ 
    Computes wind power based on wind speed input. For wind turbine model GE1.5-77.

    Parameters
    ----------
        wind_speeds: np.ndarray
            Time series of wind speeds.

    Returns
    -------
        Time series of wind power capacity factors derived from wind speeds.
    """

    power_output = np.interp(wind_speeds, rated_wind_speeds, rated_wind_power) # interpolate between measured points for wind power

    # wind power output is zero, below the cut-in speed
    power_output = np.where(wind_speeds < cut_in, 0, power_output)

    # wind power output is zero, above the cut-out speed
    power_output = np.where(wind_speeds > cut_out, 0, power_output)

    # wind power output is maxed out, between rated wind speed and cut-out speed
    power_output = np.where((wind_speeds > rated_wind) & (wind_speeds < cut_out), 1, power_output)

    # convert power outputs into capacity factor (range: 0 to 1)
    capacity_factor = power_output/rated_power

    return capacity_factor